In [1]:
!pip install ultralytics


In [2]:
import os
from ultralytics import YOLO
import psycopg2
from dotenv import load_dotenv
import pandas as pd
from PIL import Image
import logging

load_dotenv()  # Load .env database config
logging.basicConfig(level=logging.INFO)


🔷 Cell 3 – Connect to PostgreSQL

In [3]:
conn = psycopg2.connect(
    dbname=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASS"),
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT")
)
cursor = conn.cursor()


🔷 Cell 4 – Load YOLOv8 Pre-trained Model



In [4]:
model = YOLO("yolov8n.pt")  # Small & fast model; can also use yolov8s.pt or yolov8m.pt


🔷 Cell 5 – Define Image Directory
Update this path to the folder containing images downloaded in Task 1:

In [5]:
import glob

# Recursively find all .jpg and .png images inside subfolders
image_dir = r"C:\Users\Bob\Desktop\10Acadamy\week7\medical-data-warehouse\data\images"
image_paths = glob.glob(os.path.join(image_dir, "**", "*.jpg"), recursive=True)
image_paths += glob.glob(os.path.join(image_dir, "**", "*.png"), recursive=True)

logging.info(f"🖼️ Found {len(image_paths)} images in all subfolders")


INFO:root:🖼️ Found 114 images in all subfolders


🔷 Cell 6 – Detect Objects in Each Image



In [6]:
detections = []

for img_path in image_paths:
    results = model(img_path)
    boxes = results[0].boxes
    for box in boxes:
        detection = {
            "image_name": os.path.basename(img_path),
            "object_class": int(box.cls[0]),
            "confidence": float(box.conf[0])
        }
        detections.append(detection)

df_detections = pd.DataFrame(detections)
df_detections.head()



image 1/1 C:\Users\Bob\Desktop\10Acadamy\week7\medical-data-warehouse\data\images\CheMed123\photo_2022-12-22_06-40-25 (1).jpg: 640x640 1 bottle, 121.8ms
Speed: 4.7ms preprocess, 121.8ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\Bob\Desktop\10Acadamy\week7\medical-data-warehouse\data\images\CheMed123\photo_2022-12-22_06-40-25.jpg: 640x640 1 bottle, 94.8ms
Speed: 7.2ms preprocess, 94.8ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\Bob\Desktop\10Acadamy\week7\medical-data-warehouse\data\images\CheMed123\photo_2022-12-23_06-26-15 (1).jpg: 640x640 (no detections), 151.5ms
Speed: 7.3ms preprocess, 151.5ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\Bob\Desktop\10Acadamy\week7\medical-data-warehouse\data\images\CheMed123\photo_2022-12-23_06-26-15.jpg: 640x640 (no detections), 91.7ms
Speed: 5.5ms preprocess, 91.7ms inference, 0.9ms postprocess per image at shape (1, 3, 

,image_name,object_class,confidence
0,photo_2022-12-22_06-40-25 (1).jpg,39,0.286617
1,photo_2022-12-22_06-40-25.jpg,39,0.286617
2,photo_2022-12-27_17-06-32 (1).jpg,0,0.686455
3,photo_2022-12-27_17-06-32 (1).jpg,0,0.440193
4,photo_2022-12-27_17-06-32 (1).jpg,46,0.280936


🔷 Cell 7 – OPTIONAL: Map Classes to Names



In [7]:
# YOLOv8 COCO class labels
class_names = [
    'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train',
    'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep',
    'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard',
    'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
    'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork',
    'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange',
    'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
    'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv',
    'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
    'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
    'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

# Map the object class index to the label
df_detections["object_label"] = df_detections["object_class"].apply(lambda x: class_names[x])


🔷 Cell 8 – Create Table fct_image_detections in PostgreSQL



In [8]:
create_table_sql = """
CREATE TABLE IF NOT EXISTS fct_image_detections (
    id SERIAL PRIMARY KEY,
    message_id BIGINT,
    object_class INT,
    object_label TEXT,
    confidence_score FLOAT
);
"""
cursor.execute(create_table_sql)
conn.commit()
print("✅ Table 'fct_image_detections' created without foreign key constraint.")


✅ Table 'fct_image_detections' created without foreign key constraint.


🔷 Cell 9 – Insert Detection Results into PostgreSQL
Assumption: You have a way to map image_name to message_id (e.g., filenames contain message IDs).

Update this if your file naming follows a pattern like msg_12345.jpg:

In [9]:
def extract_message_id(image_name):
    try:
        return int(image_name.split("_")[1].split(".")[0])
    except:
        return None

for _, row in df_detections.iterrows():
    message_id = extract_message_id(row["image_name"])
    if message_id is None:
        continue  # Skip if ID not found

    cursor.execute("""
        INSERT INTO fct_image_detections (image_name, object_class, confidence_score, object_label, message_id)
        VALUES (%s, %s, %s, %s, %s)
    """, (
        row["image_name"],
        row["object_class"],
        row["confidence"],
        row.get("object_label", None),
        message_id
    ))

conn.commit()
print("✅ Inserted detection results into fct_image_detections.")


✅ Inserted detection results into fct_image_detections.


🔷 Cell 10 – Query Detection Table



In [10]:
cursor.execute("SELECT * FROM fct_image_detections LIMIT 10")
for row in cursor.fetchall():
    print(row)
